# Hydrostats Features
Hydrostats is a module for time series comparison. It has preproccessing tools, visualization tools, a library of over 50 error metrics for quantitative analysis of data from two timeseries, and analysis tools. In this notebook examples are given for most of the hydrostats features. The two code blocks below shows how to install hydrostats with PIP and show the current version (make sure it is the most recent version). It also imports the necessary modules for this demonstration.

In [ ]:
# Run these commands if you have not yet installed hydrostats, and make sure it is the latest version

# !pip install hydrostats --upgrade
!pip show hydrostats

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 10
import hydrostats as hs
import hydrostats.data as hd
import hydrostats.visual as hv
import matplotlib.pyplot as plt

## Preprocessing
Hydrostats has tools that simply the preprocessing of data so that an analysis can be run on two timeseries. Once the data has been preprocessed then the analysis can be run on the two time-series. 

### Merge Data Function
The first thing we need to do is load our data into the notebook, which is done below. In this example we will be comparing forecasts from the Streamflow Prediction Tool (SFPT) Tethys app to forecasts from the Global Flood Awareness System (GLOFAS). The data is loaded from the Hydrostats github repository, but if a user wanted to load local files, they would need to supply the path to the files instead. In this example the data is daily for both datasets. The merge_data Hydrostats function function used below allows a user to take two timeseries and merge them into a combined pandas dataframe.

In [ ]:
# Defining the URLs of the datasets
github_base_url = r'https://raw.github.com/BYU-Hydroinformatics/Hydrostats/master/Sample_data'
sfpt_url = github_base_url + r'/sfpt_data/magdalena-calamar_interim_data.csv'
glofas_url = github_base_url + r'/GLOFAS_Data/magdalena-calamar_ECMWF_data.csv'

merged_df = hd.merge_data(sfpt_url, glofas_url, column_names=['SFPT', 'GLOFAS'])
merged_df

### Seasonal Periods
Once we have the entire dataset merged into one combined dataframe, can find then go on to find certain seasonal periods that we want to analyze with the seasonal_period hydrostats function. In the example below, we only want to analyze from April 1st to July 31st seasonally within the time range of Jan. 1st, 1986 to Dec. 31st, 1992.

In [ ]:
seasonal_df = hd.seasonal_period(merged_df, ['04-01', '07-31'], time_range=['1986-01-01', '1992-12-31'])
seasonal_df

### Daily and Monthly Averages
We can also easily find the daily and monthly averages in streamflows over the 35 years of daily data we have. In the example below we use the daily_average function, but a monthly_average function() in the same module is also available. We create a new dataframe that contains the daily averages for this time-series below. 

In [ ]:
daily_avg_df = hd.daily_average(merged_data=merged_df)
daily_avg_df

### Daily and Monthly Standard Deviation and Standard Error
Hydrostats also can calculate daily and monthly standard deviation as well as daily and monthly standard error. We create a new dataframe below that contains the daily standard error for this time-series below.  

In [ ]:
daily_std_error = hd.daily_std_error(merged_data=merged_df)
daily_std_error

## Visualization
Once preprocessing on the data is finished, the user can then start to visualize the data with the Hydrostats visual module. There are 4 functions included in the Hydrostats visual model, which are: plot, hist, scatter, and qqplot. These functions are demonstrated below. 

### The Plot Function
The plot function will simply take the observed and simulated data and compare them by plotting them on the vertical axis with respect to time. When comparing stream flow rates, this is known as a hydrograph, but the plot function can be used for any time series if desired by the user. An few examples of using the plot function is shown below. By using this function, we are able to plot the entire time series, the seasonal period that we specified, the daily averages, and the gaily averages with error bars. 

**Note**: If you are viewing these examples in Google's Colaboratory, they will not display properly. You will need to use jupyter notebooks or jupyter lab to view them properly.

#### Plotting the Entire Time Series
The entire time series is plotted below with the hydrostats visual plot function. In this example, the linestyles are changed from the default, a title, legend, and labels are added, and metrics are calculated and displayed on the side. 

In [ ]:
import hydrostats.visual as hv

hv.plot(merged_data_df=merged_df, 
        title='Hydrograph of Entire Time Series',
        linestyles=['r-', 'k-'], 
        legend=['SFPT', 'GLOFAS'], 
        labels=['Datetime', 'Streamflow (cfs)'], 
        metrics=['ME', 'NSE', 'SA'],
        fig_size=(14, 8),
        text_adjust=(-0.25, 0.8)
       )
plt.show()

#### Plotting Seasonal Averages with Error Bars
Below, it is demonstrated how to plot daily averages with error bars. You simply need to pass in the daily average dataframe that we created earlier in the pre-processing step (daily_avg_df()) and pass in the daily standard error dataframe in the ebars argument (daily_std_error()). We change the linestyle and the color of the ebars below and a title, legend, and labels are added. 

In [ ]:
hv.plot(merged_data_df=daily_avg_df,
        title='Daily Average Streamflow (Standard Error)',
        legend=['SFPT', 'GLOFAS'],
        x_season=True,
        labels=['Datetime', 'Streamflow (csm)'],
        linestyles=['r-', 'k-'],
        fig_size=(14, 8),
        ebars=daily_std_error,
        ecolor=['r', 'k']
        )
plt.show()

### Plotting a Histogram
To plot a histogram, we can either use the merged dataframe that we created or two arrays of simulated and observed data. In this case, we use the merged dataframe because we already have it saved to memory. We specify the number of bins and then add a legend, title, and labels. We can compare the distribution of the data with this tool.

In [ ]:
hv.hist(merged_data_df=merged_df,
        num_bins=100,
        title='Histogram of Streamflows',
        legend=['SFPT', 'GLOFAS'],
        labels=['Bins', 'Frequency'])
plt.show()

### Plotting a Quantile-Quantile (qq) Plot
Similar to the histogram, the qq plot is a means to check whether two datasets come from the same distribution. In the example below, we use the merged dataframe (merged_df) to supply the data. We can also choose to use two array of both simulated and observed data if desired. We then create a title, legend, and labels. We also change the size of the figure (inches).

In [ ]:
hv.qqplot(merged_data_df=merged_df, title='Quantile-Quantile Plot of Data',
          xlabel='SFPT Data Quantiles', ylabel='GLOFAS Data Quantiles', legend=True, 
          figsize=(8, 6))
plt.show()

### Plotting a Scatter Plot
We can plot scatter plots using the scatter() function. In the example below we pass in the data in the form of arrays to demonstrate how to do this. We plot the data in a scatter plot with both a normal scale and a log-log scale. Typically, the Log-Log scale spreads out the data for a more clear representation. In this example, the data looks about the same with either scale. We add a title, labels, and a legend for the lines added.   

In [ ]:
SFPT_array = merged_df.iloc[:, 0].values
GLOFAS_array = merged_df.iloc[:, 1].values

hv.scatter(sim_array=SFPT_array, obs_array=GLOFAS_array, grid=True, title='Scatter Plot (Normal Scale)', 
           labels=['SFPT', 'GLOFAS'], best_fit=True)
plt.show()

hv.scatter(sim_array=SFPT_array, obs_array=GLOFAS_array, grid=True, title='Scatter Plot (Log-Log Scale)', 
           labels=['SFPT', 'GLOFAS'], line45=True, log_scale=True)
plt.show()

## Error Metric Calculations
AFter we have done the preprocessing and the visualization, we can do a more in depth analysis of the data by creating tables of metrics and doing a time lag analysis to check for any potential timing errors in the data. 

### Calculating Error Metric with the HydroErr package

Hydrostats has a module that pulls from the HydroErr package that allows users to access over 60 different error metrics. These metrics have data checks to make sure that the data that you are inputing to the metrics does not have NaN or Inf values, and can optionally check for zero and negative values. Below we demonstrate how to calculate error metrics, and highlight how to remove zero values.

In [ ]:
# Putting the data from the dataframe into 1D arrays
sfpt_array = merged_df.iloc[:, 0].values
glofas_array = merged_df.iloc[:, 1].values

# Calculating the KGE metric
hs.kge_2012(sfpt_array, glofas_array)

In [ ]:
# Note that when we remove zeros the indices where zeros are located are given through a warning
hs.kge_2012(sfpt_array, glofas_array, remove_zero=True)

### Creating a Table of Error Metrics

To create a table of metrics, we use the make_table() function, located in the main hydrostats import. In this function we can specify the time periods we want to analyze (the entire time series will be analyzed by default) as well as the metrics that we want to use. Below we specify the merged dataframe, the metrics we want to use, the time periods to want to evaluate (high and low periods in this case), to remove zeros and negatives, and the location of the station to put in the table. This may raise a lot of errors if the two timeseries have values that need to be removed, so if you would like to ignore the errors, the warnings.simplefilter('ignore') can be applied. 

To find what abbreviations correspond to which metric, a table is provided in the docs [here](https://waderoberts123.github.io/Hydrostats/ref_table.html).

In [ ]:
hs.make_table(merged_dataframe=merged_df, 
              metrics=['MAE', 'r2', 'ACC', 'NSE', 'KGE (2012)', 'SA'], 
              seasonal_periods=[['01-01', '03-31'], ['04-01', '06-30'], ['07-01', '09-30'], ['10-01', '12-31']], 
              remove_neg=True, remove_zero=True, 
              location='Magdalena')

## Ensemble Adjusted Error Metrics and Skill Score

Hydrostats also includes many common forecast ensemble metrics such as the ensemble mean squared error (EMSE), the continuous ranked probability score (CRPS), the area under the receiver operating characteristic curve (AUROC), and the ensemble adjusted brier score (EABS). Users can also calculate skill scores for all of these metrics comparing their model’s forecast performance to that of a benchmark forecast.

Below we give an example of calculating the crps scores for an ensemble forecast and a benchmark and then computing the skill score to compare the two forecasts.

In [ ]:
import hydrostats.ens_metrics as em 

# Generate random data as an example 
np.random.seed(6543934)
ens_array = (np.random.rand(15, 52) + 1) * 100  # Matrix 15 x 52, range [0, 100)
obs_array = (np.random.rand(15) + 1) * 100 # 1D matrix of length 15, range [0, 100)

# Compute crps metric values  
crps_forecast = em.ens_crps(obs_array, ens_array)['crps'] # Only store the scores in our variable

# Compute example benchmark forecast scores
crps_benchmark = em.ens_crps(obs_array, ens_array[:, 1:3])['crps'] # Only use first two ensembles as a benchmark

# Compute the skill score
skill_score = em.skill_score(scores=crps_forecast, bench_scores=crps_benchmark, perf_score=0)
print(skill_score)

# In this case, using the 52 ensembles is slightly more skillful than using just the first two. 
# This is random though so this does not mean much.